# Fusion Retriever
This retriever takes a straightforward approach to combining results from multiple retrievers. It simply concatenates the results from different retrievers, providing a combined set of potentially relevant information without any complex ranking or prioritization

Pros:



- Straightforward Combination: Offers a simple way to merge results from multiple retrievers by concatenating them.


Cons:



Lack of Ranking Sophistication:  May not effectively prioritize the most relevant results as it does not employ a ranking algorithm to order the merged output.

In [1]:
from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama

ollama_embedding = OllamaEmbedding(
    model_name="mxbai-embed-large",
    base_url="http://localhost:11434",
)

ollama = Ollama(
    model="llama3.2:3b-instruct-fp16",
    base_url="http://localhost:11434"
)

Settings.llm = ollama
Settings.embed_model = ollama_embedding

In [2]:
from llama_index.core import SimpleDirectoryReader

documents_1 = SimpleDirectoryReader(
    input_files=["../documents/ods-cpp (1).pdf"]
).load_data()
documents_2 = SimpleDirectoryReader(
    input_files=["../documents/RAT.pdf"]
).load_data()

In [3]:
from llama_index.core import VectorStoreIndex

index_1 = VectorStoreIndex.from_documents(documents_1)
index_2 = VectorStoreIndex.from_documents(documents_2)

In [4]:
from llama_index.core.retrievers import QueryFusionRetriever

retriever = QueryFusionRetriever(
    [index_1.as_retriever(), index_2.as_retriever()],
    similarity_top_k=2,
    num_queries=4,  # set this to 1 to disable query generation
    use_async=True,
    verbose=True,
    # query_gen_prompt="...",  # we could override the query generation prompt here
)

In [5]:
# apply nested async to run in a notebook
import nest_asyncio

nest_asyncio.apply()

In [6]:
nodes_with_scores = retriever.retrieve("What is the difference between stack and queue?")

Generated queries:
Here are three search queries based on your input:
1. How does a stack differ from a data structure that follows the FIFO (first-in-first-out) principle?
2. In computer science, what is the main distinction between using a last-in-first-out (LIFO) approach and using a queue to manage elements?
3. What are the key differences in terms of algorithmic usage and application between stacks and queues in programming?


In [7]:
for node in nodes_with_scores:
    print(f"Score: {node.score:.2f} - {node.text[:100]}...")

Score: 0.79 - Interfaces §1.2
x · · ·
add (x)/enqueue (x) remove ()/dequeue ()
Figure 1.1: A FIFO Queue .
1.2.1 Th...
Score: 0.76 - §1.2 Introduction
16add(x)remove ()/deleteMin ()
x6
133
Figure 1.2: A priority Queue .
· · ·
remove ...


In [8]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever)

In [9]:
response = query_engine.query(
    "What is the difference between stack and queue?"
)

Generated queries:
Here are three generated search queries related to your input:
1. "data structure comparison stack vs queue"
2. "programming concepts stack vs queue"
3. "algorithms difference between stack and queue"


In [10]:
from llama_index.core.response.notebook_utils import display_response

display_response(response)

**`Final Response:`** One of the main differences between a stack and a queue is how elements are accessed. In a stack, elements are added and removed from one end (usually referred to as the top or head), which involves choosing that specific index upon which to perform operations.

In contrast, for an efficient array-based implementation of a queue, it would be better to maintain two separate indices: one keeping track of the next element to remove (tail) and another counting the number of elements in the queue. This allows for more balanced and efficient access to both the front and back of the queue.